<a href="https://colab.research.google.com/github/kristenvonbecker/spotify_data_analysis/blob/main/etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spotify Data Analysis, Part 1: The ETL**

---




by Kristen Beck ([kristenvonbecker.com](https://kristenvonbecker.com))

This notebook constitutes the first phase of the [Spotify Data Analysis](https://kristenvonbecker.com/projects/spotify-data-analysis/) project: building the dataset. The code which follows can be used to:

1. Build a customized Pandas dataframe containing audio feature data for several collections (i.e. playlists) of Spotify tracks.
2. Export the dataframe in CSV format to your local machine. This file can then uploaded to the notebook for Part 2: The Analysis.

## Getting started

In order to use the interactive feature of this notebook, you *must* have a Spotify developer account and API key. If you've already done this, continue on. If not, please follow [these instructions](https://developer.spotify.com/documentation/web-api/tutorials/getting-started#create-an-app) to get a `SPOTIPY_CLIENT_ID` (like a username) and
`SPOTIPY_CLIENT_SECRET` (like a password), both of which you'll set as environment variables below.

### Install & import dependencies

In [ ]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
# for import/export in Colab
from google.colab import files
import io

# for connecting to Spotify Web API
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# for handling data
import json
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import numpy as np
from datetime import datetime
import random

### Configure and authorize Spotipy client

Set environment variables:

In [ ]:
# ENTER YOUR SPOTIFY CLIENT CREDENTIALS HERE:
%env SPOTIPY_CLIENT_ID=#yours goes here
%env SPOTIPY_CLIENT_SECRET=#yours goes here

# it's okay to leave this as is
%env SPOTIPY_REDIRECT_URI=http://localhost:8888/callback

Create Spotipy client instance:

In [ ]:
# client credentials flow
# https://spotipy.readthedocs.io/en/2.22.1/#client-credentials-flow

auth_manager = SpotifyClientCredentials()

In [ ]:
# this step initializes the Spotipy client
# in case of rate limiting, adjust values of request_timeout and retries

sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=10, retries=5)

In the code below, we'll use this Spotipy API client to request playlist- and track-level data from Spotify using one of the following endpoints: `sp.user_playlists()`, `sp.featured_playlists()`, `sp.search()`, `sp.playlists()`, `sp.tracks()`, and `sp.audio_features()`. For more information about these endpoints (or the Spotipy client module), see [the docs](https://spotipy.readthedocs.io/en/2.22.1/#module-spotipy.client).

## Defining scope of data

Before beginning to build the dataframe, we need to decide which data should be included in our study. This project is only intended to provide an overview of various data visualization techniques, so we have a lot of freedom in regards to data curation. Therefore, we'll choose our list of tracks to include in the data by simply selecting [playlists](https://support.spotify.com/us/article/find-playlists/) of interest. This approach is advantageous for two reasons:
- The `sp.playlist()` endpoint will return details (incuding track IDs) for a playlist which is specified by its playlist ID.
- A playlist is a natural grouping of tracks (according to genre, etc.), so identifying tracks in this manner introduces a way to label the data.
- Spotify maintains *thousands* of curated playlists that we can query by name using `sp.search(type='playlist')`.

As a minimum, we will include a handful playlists which are currently promoted by Spotify (and will therefore tend to include more popular tracks). The user (you!) will also have the opporutnity in this step to include additional Spotify playlists in the analysis.

### Scripts

First we define the main functions which we'll use below to build lists of track IDs for given playlist IDs.

The function `get_playlist_tracks()` takes `playlists`, a list of playlist objects (which are dictionaries,) and a `name` for the grouping and returns a dictionary whose keys are playlist names and whose values are track IDs associated to a given playlist.

In [ ]:
def get_playlist_tracks(playlists, name):
  tracks = {}

  for item in playlists:
    playlist_name = item['name'].strip()
    playlist_id = item['id']
    playlist = sp.playlist(playlist_id)
    playlist_track_ids = [x['track']['id'] for x in playlist['tracks']['items']]
    tracks[playlist_name] = playlist_track_ids

  heading = name
  print(f"{heading}")
  print("-"*len(heading))
  for playlist, track_ids in tracks.items():
    num_tracks = len(track_ids)
    print(f"{playlist}: {num_tracks} tracks")
  if not tracks:
    print("N/A")
  print()

  return tracks

The function `add_playlists()` starts an interactive console session in order to gather any additional playlists the user wishes to include in the study. Each playlist name entered is queried using the `sp.search()` endpoint. Results, ordered by relevance, are returned to the user for confirmation before new items are added.

In [ ]:
def add_playlists():

  added_playlists = []

  loop = True
  while loop:
    query = input("\n----------\n\nEnter playlist name (RETURN to quit): ")

    if not query:
      loop = False
      continue

    results = sp.search(query, type='playlist')

    for item in results['playlists']['items']:
      print("\n----------\n")
      print(f"{item['name']}: {item['description']}")
      print(f"Owner: {item['owner']['display_name']}\n")

      confirm = input("Include in analysis? (Y, N, or RETURN to quit) ")

      if confirm[0].lower() == 'y':
        added_playlists.append(item)
        break

      elif confirm[0].lower() == 'n':
        continue

      else:
        break

  print(f"\n----------\n\nRecorded {len(added_playlists)} playlist IDs\n")

  return added_playlists

### Getting the track IDs

Now we use the scripts defined above to build a dictionary which contains track IDs by playlist name.

Get track IDs for playlists returned by `sp.user_playlists('spotify')`:

In [ ]:
spotify_playlists = sp.user_playlists('spotify', limit=None)['items']
spotify_playlist_tracks = get_playlist_tracks(spotify_playlists, "Top Spotify playlists")

Top Spotify playlists
---------------------
Today's Top Hits: 50 tracks
RapCaviar: 50 tracks
Hot Country: 50 tracks
Viva Latino: 50 tracks
New Music Friday: 100 tracks
Peaceful Piano: 100 tracks
Are & Be: 50 tracks
Rock Classics: 100 tracks
mint: 75 tracks
Rock This: 50 tracks
just hits: 100 tracks
All Out 2000s: 100 tracks
All Out 90s: 100 tracks
All Out 80s: 100 tracks
All Out 70s: 100 tracks
All Out 60s: 100 tracks
All Out 50s: 100 tracks
Soft Pop Hits: 100 tracks
Signed XOXO: 100 tracks



Get track IDs for Spotify's featured playlists, returned by `sp.featured_playlists()`, which are updated daily:

In [ ]:
featured_playlists = sp.featured_playlists()['playlists']['items']
featured_playlist_tracks = get_playlist_tracks(featured_playlists, "Featured Spotify playlists")

Featured Spotify playlists
--------------------------
Afternoon Acoustic: 100 tracks
Just Good Music: 70 tracks
Indigo: 100 tracks
Pumped Pop: 75 tracks
Feel Good Classical: 49 tracks
Rock Classics: 100 tracks
Stress Relief: 100 tracks
Jazz Vibes: 100 tracks
Nu-Funk: 100 tracks
Fresh & Chill: 82 tracks



Ask the user for additional playlists to include, then get track IDs for these playlists:

In [ ]:
added_playlists = add_playlists()
added_playlist_tracks = get_playlist_tracks(added_playlists, "User-specified playlists")


----------

Enter playlist name (RETURN to quit): classical garden

----------

Classical Garden: The perfect classical tunes for pruning, planting, and putting your feet up.
Owner: Spotify

Include in analysis? (Y, N, or RETURN to quit) y

----------

Enter playlist name (RETURN to quit): melantronic

----------

Melantronic: Beautiful electronic music for melancholy moments.
Owner: Spotify

Include in analysis? (Y, N, or RETURN to quit) y

----------

Enter playlist name (RETURN to quit): music for plants

----------

Music for Plants: Mellow vibrations for green leaves, green thumbs and green ears.
Owner: Spotify

Include in analysis? (Y, N, or RETURN to quit) y

----------

Enter playlist name (RETURN to quit): 

----------

Recorded 3 playlist IDs

User-specified playlists
------------------------
Classical Garden: 51 tracks
Melantronic: 50 tracks
Music for Plants: 100 tracks



Combine data into a single dictionary of track IDs by playlist name:

In [ ]:
playlist_tracks = spotify_playlist_tracks | featured_playlist_tracks # | added_playlist_tracks

print("Joined data for:")
print(f" Top Spotify playlists ({len(spotify_playlist_tracks)})")
print(f" Featured Spotify playlists ({len(featured_playlist_tracks)})")
# print(f" User-specifified playlists ({len(added_playlist_tracks)})")
print(f"\nSaved track IDs for {len(playlist_tracks)} unique playlists")

Joined data for:
 Top Spotify playlists (19)
 Featured Spotify playlists (10)

Saved track IDs for 28 unique playlists


## Building the dataframe

Next, we use the lists of track IDs by playlist that were created in the last step (and packaged as `playlist_tracks`) in order to query batches of tracks IDs from the Spotify API using the endpoints `sp.tracks()` and `sp.audio_features()`. As track-level data is collected, cleaned, coded, and loaded into a dataframe, the relevant playlist(s) for each track are recorded in a separate field, `playlists`. For a description of the track-level fields which are collected in this step, please refer to the file *data_description.txt*.

Initialize empty data structures:

In [ ]:
keys = [
    'id',
    'name',
    'artists',
    # 'genres',
    'album',
    'images',
    'image_sizes',
    'release_dt',
    'duration',
    'popularity',
    'explicit',
    'dance',
    'energy',
    'loud',
    'speech',
    'acoustic',
    'instrument',
    'live',
    'valence',
    'tempo',
    'tempo_conf',
    'key',
    'key_conf',
    'mode',
    'mode_conf',
    'time_sig',
    'time_sig_conf',
    'playlist'
]

def init_df():
  track_df = pd.DataFrame(columns=keys)
  return track_df

def init_dict():
  track_dict = {key: [] for key in keys}
  return track_dict

Get track-level data for all tracks on playlist:

In [ ]:
def get_track_data(track_ids, playlist=None):

  num_tracks = len(track_ids)
  track_data = []
  audio_features_data = []
  audio_analysis_data = []
  num_calls = 0

  for _ in range(int(np.ceil(num_tracks/50))):
    track_results = sp.tracks(track_ids[50*num_calls:50*num_calls+50])
    track_data.extend(track_results['tracks'])

    audio_features_results = sp.audio_features(track_ids[50*num_calls:50*num_calls+50])
    audio_features_data.extend(audio_features_results)

    num_calls += 1

  track_dict = init_dict()

  for i in range(num_tracks):
    audio_analysis_results = sp.audio_analysis(track_ids[i])
    audio_analysis_data.append(audio_analysis_results['track'])

    track_dict['id'].append(track_data[i]['id'])
    track_dict['name'].append(track_data[i]['name'])
    track_dict['artists'].append([artist['name'] for artist in track_data[i]['artists']])
    track_dict['album'].append(track_data[i]['album']['name'])
    track_dict['images'].append([image['url'] for image in track_data[i]['album']['images']])
    image_sizes = zip(
          [image['height'] for image in track_data[i]['album']['images']],
          [image['width'] for image in track_data[i]['album']['images']]
        )
    track_dict['image_sizes'].append([(h, w) for h, w in image_sizes])
    track_dict['release_dt'].append(track_data[i]['album']['release_date'])
    track_dict['duration'].append(audio_analysis_data[i]['duration'])
    track_dict['popularity'].append(track_data[i]['popularity'])
    track_dict['explicit'].append(track_data[i]['explicit'])
    track_dict['dance'].append(audio_features_data[i]['danceability'])
    track_dict['energy'].append(audio_features_data[i]['energy'])
    track_dict['loud'].append(audio_features_data[i]['loudness'])
    track_dict['speech'].append(audio_features_data[i]['speechiness'])
    track_dict['acoustic'].append(audio_features_data[i]['acousticness'])
    track_dict['instrument'].append(audio_features_data[i]['instrumentalness'])
    track_dict['live'].append(audio_features_data[i]['liveness'])
    track_dict['valence'].append(audio_features_data[i]['valence'])
    track_dict['tempo'].append(audio_analysis_data[i]['tempo'])
    track_dict['tempo_conf'].append(audio_analysis_data[i]['tempo_confidence'])
    track_dict['key'].append(audio_analysis_data[i]['key'])
    track_dict['key_conf'].append(audio_analysis_data[i]['key_confidence'])
    track_dict['mode'].append(audio_analysis_data[i]['mode'])
    track_dict['mode_conf'].append(audio_analysis_data[i]['mode_confidence'])
    track_dict['time_sig'].append(audio_analysis_data[i]['time_signature'])
    track_dict['time_sig_conf'].append(audio_analysis_data[i]['time_signature_confidence'])
    track_dict['playlist'].append(playlist)

  track_df = pd.DataFrame.from_dict(track_dict)

  track_df['release_dt'] = pd.to_datetime(track_df['release_dt'])
  track_df['popularity'] = pd.to_numeric(track_df['popularity'])
  track_df['explicit'] = track_df['explicit'].astype('boolean')
  track_df['key'] = pd.Categorical(track_df['key'],
                                   categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
                                   ordered=True)
  track_df['mode'] = pd.Categorical(track_df['mode'],
                                   categories=[0, 1],
                                   ordered=True)
  track_df['time_sig'] = track_df['time_sig'].astype('category')

  return track_df

Load track-level data into a dataframe:

In [ ]:
def build_track_df():

  track_df = init_df()

  playlists = []

  for playlist in playlist_tracks.keys():

    playlists.append(playlist)

    exist_ids = [id for id in playlist_tracks[playlist] if id in track_df['id'].values]
    mask = [track_df['id'].iloc[i] in exist_ids for i in track_df.index]
    add_playlist = ["|" + playlist if item else "" for item in mask]
    track_df['playlist'] += add_playlist

    new_ids = [id for id in playlist_tracks[playlist] if id not in exist_ids]

    try:
      new_tracks = get_track_data(new_ids, playlist=playlist)

    except ConnectionError:

      try:
        new_tracks = get_track_data(new_ids)

      except ConnectionError:
        print(f"Unable to get tracks from playlist {playlist}")
        continue

    track_df = pd.concat([track_df, new_tracks], ignore_index=True)

  # playlist_dummies = track_df['playlist'].str.get_dummies()
  # track_df = pd.concat([track_df, playlist_dummies], axis=1).drop(columns=['playlist'])

  return track_df, playlists

Build Pandas dataframe object:

In [ ]:
track_df, playlists = build_track_df()

## Investigating dataframe

Print dataframe column names and datatypes:

In [ ]:
track_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2097 entries, 0 to 2096
Data columns (total 55 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2097 non-null   object        
 1   name               2097 non-null   object        
 2   artists            2097 non-null   object        
 3   album              2097 non-null   object        
 4   images             2097 non-null   object        
 5   image_sizes        2097 non-null   object        
 6   release_dt         2097 non-null   datetime64[ns]
 7   duration           2097 non-null   float64       
 8   popularity         2097 non-null   object        
 9   explicit           2097 non-null   boolean       
 10  dance              2097 non-null   float64       
 11  energy             2097 non-null   float64       
 12  loud               2097 non-null   float64       
 13  speech             2097 non-null   float64       
 14  acoustic

Print the first few rows of the dataframe:

In [ ]:
track_df.head()

,id,name,artists,album,images,image_sizes,release_dt,duration,popularity,explicit,...,Rock Classics,Rock This,Signed XOXO,Soft Pop Hits,Spilled Ink,Today's Top Hits,Top Gaming Tracks,Viva Latino,just hits,mint
0,2gyxAWHebV7xPYVxqoi86f,get him back!,[Olivia Rodrigo],GUTS,[https://i.scdn.co/image/ab67616d0000b273e8525...,"[(640, 640), (300, 300), (64, 64)]",2023-09-08,211.141,80,True,...,0,0,0,0,0,1,0,0,1,0
1,2IGMVunIBsBLtEQyoI1Mu7,Paint The Town Red,[Doja Cat],Paint The Town Red,[https://i.scdn.co/image/ab67616d0000b2737acee...,"[(640, 640), (300, 300), (64, 64)]",2023-08-04,231.750,99,True,...,0,0,1,0,0,1,0,0,1,0
2,1vYXt7VSjH9JIM5oRRo7vA,Dance The Night - From Barbie The Album,[Dua Lipa],Dance The Night (From Barbie The Album),[https://i.scdn.co/image/ab67616d0000b2737dd3b...,"[(640, 640), (300, 300), (64, 64)]",2023-05-25,176.579,97,False,...,0,0,0,0,0,1,0,0,1,0
3,1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,[Taylor Swift],Lover,[https://i.scdn.co/image/ab67616d0000b273e787c...,"[(640, 640), (300, 300), (64, 64)]",2019-08-23,178.427,100,False,...,0,0,0,0,0,1,0,0,0,0
4,7x9aauaA9cu6tyfpHnqDLo,Seven (feat. Latto) (Explicit Ver.),"[Jung Kook, Latto]",Seven (feat. Latto),[https://i.scdn.co/image/ab67616d0000b273bf5cc...,"[(640, 640), (300, 300), (64, 64)]",2023-07-14,184.400,99,True,...,0,0,0,0,0,1,0,0,0,0


## Exporting the data

Export dataframe to CSV format, and save locally on your machine:

In [ ]:
track_df.to_csv('default_track_data.csv', index=False)
files.download('default_track_data.csv')

with open('default_playlists.json', 'w') as f:
        json.dump(playlists, f, indent=2)

files.download('default_playlists.json')

dt = datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")

with open('default_data_timestamp.txt', 'w') as f:
        f.write(dt)

files.download('default_data_timestamp.txt');

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

You're now ready to move on to the next notebook, Part 2: The Analysis.